In [ ]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN

video_path = "./img/22.mp4"  # <-- replace

lk_params = dict(winSize=(15, 15),
                maxLevel=3,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

motion_threshold = 1.0   # pixels
dbscan_eps = 30
dbscan_min_samples = 1
max_lost_frames = 3      # how many frames to keep box without movement

# Track clusters
clusters = {}  # {id: {"box":(x1,y1,x2,y2), "lost":0}}
next_id = 0

def iou(boxA, boxB):
    # Intersection-over-Union for box matching
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union = areaA + areaB - inter
    return inter / union if union > 0 else 0

# Open video
cap = cv2.VideoCapture(video_path)
ret, old_frame = cap.read()
if not ret:
    raise ValueError("Cannot read first frame")

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=500, qualityLevel=0.01, minDistance=5)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is None:
        break

    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Moving points
    motion = np.linalg.norm(good_new - good_old, axis=1)
    moving_points = good_new[motion > motion_threshold]

    new_boxes = []
    if len(moving_points) > 0:
        clustering = DBSCAN(eps=dbscan_eps, min_samples=dbscan_min_samples).fit(moving_points)
        labels = clustering.labels_
        for label in set(labels):
            if label == -1:
                continue
            pts = moving_points[labels == label]
            x_min, y_min = pts.min(axis=0)
            x_max, y_max = pts.max(axis=0)
            
            w = x_max - x_min
            h = y_max - y_min
            side = int(max(w, h))
            side = int(side * 1.25)
            
            cx = int((x_min + x_max) / 2)
            cy = int((y_min + y_max) / 2)
            
            x1 = cx - side // 2
            y1 = cy - side // 2
            x2 = cx + side // 2
            y2 = cy + side // 2
            
            new_boxes.append((x1, y1, x2, y2))

    # Match new boxes to existing clusters
    matched_ids = set()
    for box in new_boxes:
        best_id, best_iou = None, 0
        for cid, data in clusters.items():
            i = iou(box, data["box"])
            if i > best_iou:
                best_id, best_iou = cid, i
        if best_iou > 0.2:  # IoU threshold
            clusters[best_id]["box"] = box
            clusters[best_id]["lost"] = 0
            matched_ids.add(best_id)
        else:
            clusters[next_id] = {"box": box, "lost": 0}
            matched_ids.add(next_id)
            next_id = 1

    # Increment lost counter for unmatched clusters
    for cid in list(clusters.keys()):
        if cid not in matched_ids:
            clusters[cid]["lost"] += 1
            if clusters[cid]["lost"] > max_lost_frames:
                del clusters[cid]

    # Draw
    for cid, data in clusters.items():
        x1, y1, x2, y2 = data["box"]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {cid}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    cv2.imshow("Stable Moving Clusters", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update references
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()
